## 3. frontal 이미지 에서의 resnet으로 돌리고 성능 지표 분석(acc,recall,precision,f1-score)


### 1.데이터 구조를 확인

In [2]:
import pandas as pd

# Load the uploaded CSV files to check their structure and contents
file_path_projections = "C:/Users/PC/desktop/바탕 화면/archive/indiana_projections.csv"
file_path_reports ="C:/Users/PC/desktop/바탕 화면/archive/indiana_reports.csv"

# Read the CSV files
data_projections = pd.read_csv(file_path_projections)
data_reports = pd.read_csv(file_path_reports)

# Display the first few rows of each dataset to understand their structure
data_projections.head(), data_reports.head()


(   uid                filename projection
 0    1  1_IM-0001-4001.dcm.png    Frontal
 1    1  1_IM-0001-3001.dcm.png    Lateral
 2    2  2_IM-0652-1001.dcm.png    Frontal
 3    2  2_IM-0652-2001.dcm.png    Lateral
 4    3  3_IM-1384-1001.dcm.png    Frontal,
    uid                                               MeSH  \
 0    1                                             normal   
 1    2  Cardiomegaly/borderline;Pulmonary Artery/enlarged   
 2    3                                             normal   
 3    4  Pulmonary Disease, Chronic Obstructive;Bullous...   
 4    5  Osteophyte/thoracic vertebrae/multiple/small;T...   
 
                                             Problems  \
 0                                             normal   
 1                      Cardiomegaly;Pulmonary Artery   
 2                                             normal   
 3  Pulmonary Disease, Chronic Obstructive;Bullous...   
 4                         Osteophyte;Thickening;Lung   
 
                       

### "Frontal" 이미지에 대한 데이터 필터링 및 병합

In [3]:
# Filter for "Frontal" images in the projections dataset
frontal_data = data_projections[data_projections['projection'] == 'Frontal']

# Merge the frontal data with the reports data using 'uid' as the key
merged_data = pd.merge(frontal_data, data_reports, on='uid', how='inner')

# Check the resulting merged data
merged_data.head()


,uid,filename,projection,MeSH,Problems,image,indication,comparison,findings,impression
0,1,1_IM-0001-4001.dcm.png,Frontal,normal,normal,Xray Chest PA and Lateral,Positive TB test,None.,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.
1,2,2_IM-0652-1001.dcm.png,Frontal,Cardiomegaly/borderline;Pulmonary Artery/enlarged,Cardiomegaly;Pulmonary Artery,"Chest, 2 views, frontal and lateral",Preop bariatric surgery.,None.,Borderline cardiomegaly. Midline sternotomy XX...,No acute pulmonary findings.
2,3,3_IM-1384-1001.dcm.png,Frontal,normal,normal,Xray Chest PA and Lateral,"rib pain after a XXXX, XXXX XXXX steps this XX...",NaN,NaN,"No displaced rib fractures, pneumothorax, or p..."
3,4,4_IM-2050-1001.dcm.png,Frontal,"Pulmonary Disease, Chronic Obstructive;Bullous...","Pulmonary Disease, Chronic Obstructive;Bullous...","PA and lateral views of the chest XXXX, XXXX a...",XXXX-year-old XXXX with XXXX.,None available,There are diffuse bilateral interstitial and a...,1. Bullous emphysema and interstitial fibrosis...
4,5,5_IM-2117-1003002.dcm.png,Frontal,Osteophyte/thoracic vertebrae/multiple/small;T...,Osteophyte;Thickening;Lung,Xray Chest PA and Lateral,Chest and nasal congestion.,NaN,The cardiomediastinal silhouette and pulmonary...,No acute cardiopulmonary abnormality.


### 병합된 데이터에서 이진 레이블을 생성
#### -'problems' 열의 값에 따라 'label' 열에 이진 레이블을 생성
#### -'normal'인 경우에는 0을, 그 외의 값에는 1을 할당

In [4]:
# 이미지 파일 경로 설정
image_base_path = r'C:/Users/PC/desktop/바탕 화면/images_normalized'

# 'merged_data' 예시 (프론탈 이미지의 레이블이 포함된 데이터프레임)
# 'Problems' 열에서 'normal'이면 0, 그렇지 않으면 1로 레이블 생성
merged_data['label'] = merged_data['Problems'].apply(lambda x: 0 if x.strip().lower() == 'normal' else 1)

# 레이블 분포 확인
label_distribution = merged_data['label'].value_counts()

### 이미지 파일을 로드하고 전처리

In [5]:
import os
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# 이미지 파일 경로 설정
image_base_path = r'C:/Users/PC/desktop/바탕 화면/images_normalized'

# 'merged_data' 예시 (프론탈 이미지의 레이블이 포함된 데이터프레임)
# 'Problems' 열에서 'normal'이면 0, 그렇지 않으면 1로 레이블 생성
merged_data['label'] = merged_data['Problems'].apply(lambda x: 0 if x.strip().lower() == 'normal' else 1)

# 'merged_data'에서 모든 파일명 추출
frontal_filenames = merged_data['filename'].tolist()

# 디렉토리 내 모든 이미지 파일 로드
def load_images_from_directory(directory):
    images = []
    for file_name in os.listdir(directory):
        file_path = os.path.join(directory, file_name)
        if file_name.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif')):  # 지원하는 이미지 확장자
            try:
                image = Image.open(file_path)
                images.append((file_name, image))  # 파일명과 이미지 객체 저장
            except Exception as e:
                print(f"Error loading {file_name}: {e}")
    return images

# 이미지 불러오기
images = load_images_from_directory(image_base_path)

# 'merged_data'에 있는 파일명만 선별
frontal_images = [img for img in images if img[0] in frontal_filenames]

# 'frontal' 이미지를 처리하고 레이블을 추출
processed_images = []
labels = []

# 'frontal' 이미지 목록 출력 (파일명을 확인)
print(f"Found {len(frontal_images)} frontal images.")
for file_name, image in frontal_images:
    # 이미지를 64x64 크기로 리사이즈
    image_resized = image.resize((64, 64))
    # 이미지를 numpy 배열로 변환 (RGB -> numpy array)
    image_array = np.array(image_resized)
    
    # RGB 채널을 맞추기 위해 채널 차원 추가 (64, 64, 3 형태로)
    if len(image_array.shape) == 2:  # 흑백 이미지일 경우
        image_array = np.stack([image_array] * 3, axis=-1)  # 3채널로 변환
    
    processed_images.append(image_array)
    
    # 'merged_data'에서 파일명에 해당하는 레이블 추출
    file_name_without_path = os.path.basename(file_name)  # 경로 제외 파일명만 추출
    matching_row = merged_data[merged_data['filename'].str.lower() == file_name_without_path.lower()]
    
    if not matching_row.empty:
        label = matching_row['label'].values[0]  # 해당 파일명의 레이블 추출
        labels.append(label)
    else:
        print(f"No label found for {file_name_without_path}")



Found 3818 frontal images.


In [6]:
# 이미지와 레이블을 numpy 배열로 변환
X = np.array(processed_images)
y = np.array(labels)

# RGB 이미지는 3채널로 설정되어 있으므로 X의 형태는 (샘플 수, 64, 64, 3)
print(f"X shape: {X.shape}, y shape: {y.shape}")

# 훈련 세트와 테스트 세트로 나누기 (70% 훈련, 30% 테스트)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shapes of the datasets
X_train.shape, X_test.shape, y_train.shape, y_test.shape

X shape: (3818, 64, 64, 3), y shape: (3818,)


((3054, 64, 64, 3), (764, 64, 64, 3), (3054,), (764,))

In [7]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model

# Define the ResNet model
input_shape = (64,64, 3)
base_model = ResNet50(weights=None, include_top=False, input_tensor=Input(shape=input_shape))

# Add custom layers on top of the base model
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

# Final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 70, 70, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 32, 32, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 32, 32, 64)   256         ['conv1_conv[0][0]']             
                                                                                              

#### 정의된 신경망 모델을 10 에포크 동안 학습시키고, 학습 과정에서의 손실(loss) 및 정확도(accuracy) 등의 기록을 출력

In [8]:
model_history=model.fit(x=X_train, y=y_train, epochs=30, batch_size=32,validation_data= (X_test,y_test))
y_pred = model.predict(X_test)

Epoch 1/30
96/96 [==============================] - 148s 1s/step - loss: 1.2636 - accuracy: 0.6018 - val_loss: 0.6740 - val_accuracy: 0.6479
Epoch 2/30
96/96 [==============================] - 111s 1s/step - loss: 0.6348 - accuracy: 0.6536 - val_loss: 0.6464 - val_accuracy: 0.6479
Epoch 3/30
96/96 [==============================] - 112s 1s/step - loss: 0.6069 - accuracy: 0.6555 - val_loss: 0.6076 - val_accuracy: 0.6479
Epoch 4/30
96/96 [==============================] - 113s 1s/step - loss: 0.5812 - accuracy: 0.6640 - val_loss: 0.6260 - val_accuracy: 0.6479
Epoch 5/30
96/96 [==============================] - 89s 928ms/step - loss: 0.5848 - accuracy: 0.6375 - val_loss: 0.6717 - val_accuracy: 0.5955
Epoch 6/30
96/96 [==============================] - 73s 765ms/step - loss: 0.5667 - accuracy: 0.6938 - val_loss: 0.6087 - val_accuracy: 0.6401
Epoch 7/30
96/96 [==============================] - 75s 779ms/step - loss: 0.5690 - accuracy: 0.6817 - val_loss: 0.5995 - val_accuracy: 0.6571
Epoch 8

In [10]:

import numpy as np

# 이미지 크기를 (64, 64)로 맞추기
X_train_resized = np.array([tf.image.resize(img, (64, 64)).numpy() for img in X_train])
X_test_resized = np.array([tf.image.resize(img, (64, 64)).numpy() for img in X_test])

# Resized 데이터 확인
X_train_resized.shape, X_test_resized.shape

# 3. 모델 예측 및 임계값 조정
y_pred_probs = model.predict(X_test_resized)
threshold = 0.7  # 임계값 설정
y_pred = (y_pred_probs > threshold).astype('int32')


# 4. 성능 지표 계산
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# 결과 출력
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1-Score: {f1:.4f}")


24/24 [==============================] - 3s 102ms/step
Accuracy: 0.6387
Recall: 0.7192
Precision: 0.7221
F1-Score: 0.7206


----
 confusion matrix

In [11]:
#Accuracy of the predicted values
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.49      0.49      0.49       269
           1       0.72      0.72      0.72       495

    accuracy                           0.64       764
   macro avg       0.60      0.60      0.60       764
weighted avg       0.64      0.64      0.64       764

[[132 137]
 [139 356]]
